# Chapter 12: Database Reflection using the `Automap` Extension

Note: you will need data from chapter 10.

Necessary imports:

In [1]:
import logging

from sqlalchemy import Column, Integer, Table, select, create_engine
from sqlalchemy.ext.automap import AutomapBase, automap_base
from sqlalchemy.orm import Session, sessionmaker

Control logs with this flag (disabled):

In [2]:
DEBUG = False

In [3]:
if not DEBUG:
    logging.disable(logging.INFO)

Session maker factory:

In [4]:
DATABASE_URL = "sqlite+pysqlite:///store.db"

In [5]:
engine = create_engine(
    DATABASE_URL,
    echo=True,
)

In [6]:
SessionMaker = sessionmaker(
    bind=engine,
)

In [7]:
session = SessionMaker()

## Database reflection using the `Automap` extension:

In [8]:
Base: AutomapBase = automap_base()

In [9]:
Base.prepare(autoload_with=engine)  # reflect database

Class reflected (by default, the class names are the table names:):

In [10]:
Base.classes.keys()

['order_detail', 'employee', 'order', 'customer', 'product']

List order details using the extracted order class:

In [11]:
Order = Base.classes.order

Print the content of an order:

In [12]:
order = session.scalars(select(Order).filter_by(order_id=1)).one()
print(f"# Order: #{order.order_id}")

# Order: #1


Collection-based relationships are by default named "<classname>_collection":

In [13]:
for order_detail in order.order_detail_collection:
    quantity = order_detail.quantity
    product_name = order_detail.product.product_name
    print(f"{product_name} x{quantity}")

phone x1
phone screen protector x1
headphone x1


## Customizing Naming Rules

The "inflect" library is used to generate correct plural forms:

In [14]:
import re
import inflect  # pip install inflect

This function produces camel-cased class names and is assigned to the
`classname_for_table` parameter in `AutomapBase.prepare()`:

In [15]:
def camelize_classname(base, tablename, table):
    """
    Produce a 'camelized' class name, e.g.,
    'words_and_underscores' is transformed into 'WordsAndUnderscores'.
    """
    return str(tablename[0].upper() +
               re.sub(r'_([a-z])', lambda m: m.group(1).upper(), tablename[1:]))

This function is assigned to the `name_for_collection_relationship` parameter of
`AutomapBase.prepare()` and is used for un-camelizing and pluralizing class
names to produce relationship names for collections:

In [16]:
_pluralizer = inflect.engine()

In [17]:
def pluralize_collection(base, local_cls, referred_cls, constraint):
    """
    Produce an 'uncamelized', 'pluralized' class name, e.g.,
    'SomeTerm' becomes 'some_terms'.
    """
    referred_name = referred_cls.__name__
    uncamelized = re.sub(r'[A-Z]',
                         lambda m: "_%s" % m.group(0).lower(),
                         referred_name)[1:]
    pluralized = _pluralizer.plural(uncamelized)
    return pluralized

Now it's time to apply these custom naming functions:

In [18]:
Base: AutomapBase = automap_base()

In [19]:
Base.prepare(
    autoload_with=engine,
    classname_for_table=camelize_classname,
    name_for_collection_relationship=pluralize_collection
)

In [20]:
Base.classes.keys()

['OrderDetail', 'Employee', 'Order', 'Customer', 'Product']

Repeat the previous example with naming customization:

In [21]:
order = session.scalars(
    select(Base.classes.Order).filter_by(order_id=1)
).one()
print(f"# Order: #{order.order_id}")
for order_detail in order.order_details:
    quantity = order_detail.quantity
    product_name = order_detail.product.product_name
    print(f"{product_name} x{quantity}")

# Order: #1
phone x1
phone screen protector x1
headphone x1


## Reflecting a View

For reflection to work on the view, its primary key must be specified.

In [22]:
create_view = """
CREATE VIEW manager AS
SELECT
    employee_id AS id,
    "name",
    hire_date
FROM
    employee e
WHERE
    e.is_manager = TRUE;
"""

In [23]:
from sqlalchemy import text

In [24]:
session.execute(text(create_view))

Reflecting a view with `Automap`:

In [25]:
Base: AutomapBase = automap_base()

Add table (view) to metadata:

In [26]:
Table(
    "manager",
    Base.metadata,
    # PK is required for reflection to work
    Column('id', Integer, primary_key=True),
    autoload_with=engine,
)

Table('manager', MetaData(), Column('id', Integer(), table=<manager>, primary_key=True, nullable=False), Column('name', VARCHAR(length=127), table=<manager>), Column('hire_date', DATE(), table=<manager>), schema=None)

In [27]:
Base.prepare(
    # autoload_with=engine,
    classname_for_table=camelize_classname,
    name_for_collection_relationship=pluralize_collection
)

Classes reflected:

In [28]:
Base.classes.keys()

['Manager']

In [29]:
Manager = Base.classes.Manager

List managers:

In [30]:
with Session(engine) as session:
    for manager in session.scalars(select(Manager)):
        print(manager.id, manager.name, manager.hire_date)

1 Alice 2024-03-28
4 Louis 2024-02-28


Delete view:

In [31]:
delete_view = "DROP VIEW manager;"

In [32]:
session.execute(text(delete_view))

Remember to close the session:

In [33]:
session.close()